In [1]:
import cv as cv 
from PIL import Image
import matplotlib.pyplot as plt 
import numpy 
import pytesseract


In [2]:
image_file='data/index_02.JPG'
img=cv.imread(image_file)

In [9]:
def preprocess(image):
    image_gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    cv.imwrite('temp_2/image_gray.jpg',image_gray)
    image_blur=cv.GaussianBlur(image_gray,(7,7),0)
    cv.imwrite('temp_2/image_blur.jpg',image_blur)
    image_tresh=cv.threshold(image_blur,0,255,cv.THRESH_BINARY_INV + cv.THRESH_OTSU)[1]
    cv.imwrite('temp_2/image_thresh.jpg',image_tresh)#binariser l'image (seuil tresh=0)
    kernal = cv.getStructuringElement(cv.MORPH_RECT, (3, 13))
    cv.imwrite("temp/index_kernal.png", kernal)
    dilate = cv.dilate(image_tresh, kernal, iterations=1)
    cv.imwrite("temp/index_dilate.png", dilate)
    cnts = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=lambda x: cv.boundingRect(x)[0])
    return cnts


In [10]:
cnts=preprocess(img)

In [14]:
results  = []
for c in cnts:
    x, y, w, h = cv.boundingRect(c)
    if h > 200 and w > 20:
        roi = img[y:y+h, x:x+h]
        cv.imwrite('temp_2/index_chaque_partie.png',roi)
        cv.rectangle(img, (x, y), (x+w, y+h), (36, 255, 12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")
        print(ocr_result)
        for item in ocr_result:
            results.append(item)
cv.imwrite("temp/index_bbox_new.png", img)

['A.v. E.', '', 'Abacuc propheta 36,5. 319, 29. 373, 23.', '', 'Abagarus rex Edessenorum 382, 13.', '', 'Abdenago 585, 3.', '', 'Abel 26, 11.', '', 'Abigail 606, 6.', '', 'Abraham 26,18, 124,18. 133, 23.', '', '264. 302, 13. 303, 13. 320, 40.', '', '496, 22.', '', 'Absalon 102, 5.', '', 'Acgfredi v. Ecgtridus.', '', 'Achab rex 504, 1.', '', 'Achaz rex 504, 1.', '', 'Achiel 606, 19.', '', 'Achilon 496, 26.', '', 'Adal-, Adel-, Adhel-, Aedel-, Aed', 'Aedil-, Aedil- Edel-, Edil-, Ath', '', 'Ethel-.', 'Aedelbaldusabb.S. Petri Wiremuthen-;', 'sis et S. Pauli Girwensis 110, 28.', "Aedilberctus, Aedilberthus ep. Ha-'", 'gulstadensis, antea ep. Candidae|', 'casae sive Witernensis 27,43. 72,16.', 'Adalbertus, Aedilberctus sive Magus,’', 'discipulus et capellanus Arnonis', 'archiep. Salisburg., postea_abb.', 'Ferrariensis 254, 1. 320, 41. 322, 19.)', '416. 418, 14. 422, 10.', 'Adalpertabb.[Tegernseensis ?]497,23..', 'Adalbertus presb. monasterii 8. Mar-', 'tini Turonensis 399, 3.', '', 'Aedilbur

True

In [15]:
print(results)

['A.v. E.', '', 'Abacuc propheta 36,5. 319, 29. 373, 23.', '', 'Abagarus rex Edessenorum 382, 13.', '', 'Abdenago 585, 3.', '', 'Abel 26, 11.', '', 'Abigail 606, 6.', '', 'Abraham 26,18, 124,18. 133, 23.', '', '264. 302, 13. 303, 13. 320, 40.', '', '496, 22.', '', 'Absalon 102, 5.', '', 'Acgfredi v. Ecgtridus.', '', 'Achab rex 504, 1.', '', 'Achaz rex 504, 1.', '', 'Achiel 606, 19.', '', 'Achilon 496, 26.', '', 'Adal-, Adel-, Adhel-, Aedel-, Aed', 'Aedil-, Aedil- Edel-, Edil-, Ath', '', 'Ethel-.', 'Aedelbaldusabb.S. Petri Wiremuthen-;', 'sis et S. Pauli Girwensis 110, 28.', "Aedilberctus, Aedilberthus ep. Ha-'", 'gulstadensis, antea ep. Candidae|', 'casae sive Witernensis 27,43. 72,16.', 'Adalbertus, Aedilberctus sive Magus,’', 'discipulus et capellanus Arnonis', 'archiep. Salisburg., postea_abb.', 'Ferrariensis 254, 1. 320, 41. 322, 19.)', '416. 418, 14. 422, 10.', 'Adalpertabb.[Tegernseensis ?]497,23..', 'Adalbertus presb. monasterii 8. Mar-', 'tini Turonensis 399, 3.', '', 'Aedilbur

In [16]:
entities = []
for item in results:
    item = item.strip().replace("\n", "") 
    item = item.split(" ")[0]
    if len(item) > 2: 
        if item[0] == "A" and "-" not in item: 
            item = item.split(".")[0].replace(",", "").replace(";", "")
            entities.append(item)

In [18]:
entities=list(set(entities))#evite doublons 
print(entities)

['Adalgarius', 'A', 'Achab', 'Adriaticum', 'Abacuc', 'Abigail', 'Adelgisa', 'Aeneades', 'Abraham', 'Adalwinus', 'Adalpertabb', 'Adelperga', 'Adaula', 'Africa', 'Aegil', 'Aelim', 'Aeine', 'Aelfvaldus', 'Adalmondus', 'Aelberhtus', 'Aginus', 'Aedilberctus', 'Adoredus', 'Adrianus', 'Adalbertus', 'Aedilburga', 'Aesculapius', 'Adalhardus', 'Aedilthyda', 'Absalon', 'Abagarus', 'Aethiopia', 'Aaron', 'Aedelwaldus', 'Aethelhardus', 'Aedelhardus', 'Agathensis', 'Achiel', 'Aegyptus', 'Abdenago', 'Aegyptiacascola', 'Achilon', 'Agareni', 'Aegesippus', 'Aethiluuinus', 'Aedelbaldusabb', 'Adhelricus', 'Adalpert', 'Aethelfleda', 'Aethilredus', 'Aesne', 'Africanus', 'Abel', 'Achaz', 'Acgfredi', 'Adam', 'Agar', 'Adalgisus', 'Aedelredus']
